# 3 Autogen - Azure AI Agent integration

The AutoGen ecosystem provides everything you need to create AI agents, especially multi-agent workflows -- framework, developer tools, and applications.

The framework uses a layered and extensible design. Layers have clearly divided responsibilities and build on top of layers below. This design enables you to use the framework at different levels of abstraction, from high-level APIs to low-level components.

Core API implements message passing, event-driven agents, and local and distributed runtime for flexibility and power. It also support cross-language support for .NET and Python.
AgentChat API implements a simpler but opinionated API for rapid prototyping. This API is built on top of the Core API and is closest to what users of v0.2 are familiar with and supports common multi-agent patterns such as two-agent chat or group chats.
Extensions API enables first- and third-party extensions continuously expanding framework capabilities. It support specific implementation of LLM clients (e.g., OpenAI, AzureOpenAI), and capabilities such as code execution.

<img src="https://github.com/microsoft/autogen/raw/main/autogen-landing.jpg" width=400>

> https://microsoft.github.io/autogen/stable/<br>
> https://github.com/microsoft/autogen

In [1]:
import datetime
import autogen
import openai
import os
import sys

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from dotenv import load_dotenv
from azure.ai.projects.models import BingGroundingTool

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Python version = {sys.version}")
print(f"AutoGen version = {autogen.__version__}")
print(f"OpenAI version: {openai.__version__}")

Python version = 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
AutoGen version = 0.9.1post0
OpenAI version: 1.79.0


In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 21-May-2025 11:48:09


## Settings

In [5]:
load_dotenv("azure.env")

model = "gpt-4o"
model_deploymenyt = "gpt-4o"
connection_name = "groundingbingsearch"  # Yoiu must define a grouding bing search connection in Azure AI Foundry

In [6]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [7]:
client = AzureOpenAIChatCompletionClient(
    azure_deployment=model_deploymenyt,
    api_version="2024-05-01-preview",
    model = model,
    temperature = 0.7,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider,
)

In [8]:
async def web_ai_agent(query: str) -> str:
    print("***** Bing for Azure AI Agent Service\n")
    
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
    )
       
    with project_client:
        bing_connection = project_client.connections.get(
            connection_name=connection_name
        )
    
        bing = BingGroundingTool(connection_id=bing_connection.id)
    
        agent = project_client.agents.create_agent(
            model=model,
            name="web-agent",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"1. Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"2. Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"3. Message: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"4. Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        result = messages["data"][0]["content"][0]["text"]["value"]
  
    return result

In [9]:
bing_search_agent = AssistantAgent(
    name="bing_agent",
    model_client=client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [10]:
async def main_agent(query) -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content=query, source="user")],
            cancellation_token=CancellationToken(),
    )
    print("\nFull response:", response.chat_message)
    print("\033[1;31;34m")
    print("Answer:", response.chat_message.content)

    print("\033[1;31;32m")
    if response.chat_message.type == "TextMessage":
        print("Note: 🚫 No Bing call search was made")
    elif response.chat_message.type == "ToolCallSummaryMessage":
        print("Note: 🔍 A Bing call search was made")
    else:
        print("❓NA")

## Simple exemple with using the bing addon

In [11]:
# using  the agent that will always do a web search
await web_ai_agent("hello") 

***** Bing for Azure AI Agent Service

1. Created agent, ID: asst_47ZmBpaGjM1sVO8LTNimWvW6
2. Created thread, ID: thread_6D1gRrAcM54ICzOMWdNussHF
3. Message: {'id': 'msg_vOX07hr1n5aOxfpdi6t9FgrB', 'object': 'thread.message', 'created_at': 1747828092, 'assistant_id': None, 'thread_id': 'thread_6D1gRrAcM54ICzOMWdNussHF', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}
4. Run finished with status: completed


'Hello! How can I assist you today?'

In [12]:
# the main agent will use the llm
await main_agent("Hello")


Full response: source='bing_agent' models_usage=RequestUsage(prompt_tokens=52, completion_tokens=12) metadata={} content='Hi there! How can I assist you today?' type='TextMessage'

Answer: Hi there! How can I assist you today?

Note: 🚫 No Bing call search was made


In [17]:
# the main agent will use the llm
await main_agent("What is a LLM?")


Full response: source='bing_agent' models_usage=RequestUsage(prompt_tokens=804, completion_tokens=107) metadata={} content='An LLM, or Large Language Model, is a type of artificial intelligence algorithm designed to understand and generate human-like text. These models are trained on vast amounts of text data and use machine learning techniques to predict the likelihood of a word or sentence given the preceding context. This allows them to perform a range of natural language processing tasks such as translation, summarization, conversation, and more. Popular examples include GPT (Generative Pre-trained Transformer) models, which are designed to generate text that is coherent and contextually relevant.' type='TextMessage'

Answer: An LLM, or Large Language Model, is a type of artificial intelligence algorithm designed to understand and generate human-like text. These models are trained on vast amounts of text data and use machine learning techniques to predict the likelihood of a word 

In [18]:
# The main agent will use the bing search
await main_agent("What are the top cities in France?")


Full response: source='bing_agent' models_usage=RequestUsage(prompt_tokens=926, completion_tokens=324) metadata={} content='Here are some of the top cities in France:\n\n1. **Paris**: Known for its iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral, Paris is a hub for art, fashion, gastronomy, and culture.\n\n2. **Marseille**: Located on the Mediterranean coast, Marseille is famous for its port, multicultural atmosphere, and sites like the Basilica of Notre-Dame de la Garde.\n\n3. **Lyon**: Recognized for its cuisine and historical architecture, Lyon is a UNESCO World Heritage site situated between the Rhône and Saône rivers.\n\n4. **Toulouse**: Known as "La Ville Rose" for its terracotta buildings, Toulouse is a center for the aerospace industry with a vibrant cultural scene.\n\n5. **Nice**: Situated on the French Riviera, Nice is renowned for its beaches, Promenade des Anglais, and arts scene.\n\n6. **Bordeaux**: Famous for wine production, Bordeaux feat

In [15]:
# The main agent will use the bing search
await main_agent("What is the weather for Paris?")

***** Bing for Azure AI Agent Service

1. Created agent, ID: asst_J5X99TKBrlHHLEL81ub23kRQ
2. Created thread, ID: thread_REtYYAY0onmzsO94s9Tgasp2
3. Message: {'id': 'msg_QZVfeTcmdyL4SNdevmThZBbX', 'object': 'thread.message', 'created_at': 1747828122, 'assistant_id': None, 'thread_id': 'thread_REtYYAY0onmzsO94s9Tgasp2', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'current weather in Paris', 'annotations': []}}], 'attachments': [], 'metadata': {}}
4. Run finished with status: completed

Full response: source='bing_agent' models_usage=None metadata={} content='The current weather in Paris on May 21, 2025, is characterized by temperatures ranging from 11°C to 21°C. The skies are mostly cloudy with expected drizzle in the evening【3:0†source】【3:2†source】.' type='ToolCallSummaryMessage'

Answer: The current weather in Paris on May 21, 2025, is characterized by temperatures ranging from 11°C to 21°C. The skies are mostly cloudy with expected drizzle in the ev

In [16]:
# The main agent will use the bing search
await main_agent("Who is the prime minister of France?")

***** Bing for Azure AI Agent Service

1. Created agent, ID: asst_xhaf1EWFbA1tf4DEsn2uNrF4
2. Created thread, ID: thread_sWjy3JnDrnYZFo8V95iNFHjL
3. Message: {'id': 'msg_ZouXv6kuIEQ0Rk4ayNY04BvG', 'object': 'thread.message', 'created_at': 1747828133, 'assistant_id': None, 'thread_id': 'thread_sWjy3JnDrnYZFo8V95iNFHjL', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'current Prime Minister of France', 'annotations': []}}], 'attachments': [], 'metadata': {}}
4. Run finished with status: completed

Full response: source='bing_agent' models_usage=None metadata={} content='The current Prime Minister of France is François Bayrou. He was appointed to this position in December 2024【3:9†source】.' type='ToolCallSummaryMessage'

Answer: The current Prime Minister of France is François Bayrou. He was appointed to this position in December 2024【3:9†source】.

Note: 🔍 A Bing call search was made
